In [ ]:
import asyncio
import time
import requests
from indy import anoncreds, did, ledger, pool, wallet, IndyError

import json

async def run():
    print("Send credential offer to device -> started")

    # Set protocol version 2 to work with Indy Node 1.4
    await pool.set_protocol_version(2)

    pool_ = {
        'name': 'pool1',
        'config': json.dumps({"genesis_txn": '/home/indy/sandbox/pool_transactions_genesis'})
    }
    print("Open Pool Ledger: {}".format(pool_['name']))

    try:
        await pool.create_pool_ledger_config(pool_['name'], pool_['config'])
    except IndyError as ex:
        if ex.error_code == ErrorCode.PoolLedgerConfigAlreadyExistsError:
            pass
    pool_['handle'] = await pool.open_pool_ledger(pool_['name'], None)

    print("==============================")
    print("== Getting Trust Anchor credentials - Supplier getting Verinym  ==")
    print("------------------------------")
    
    client = MongoClient(host='supplier_db', port=27017, username='root', password='pass', authSource="admin")
    db = client["supplier_db"]

    verinym_for = db.supplier_tb.find_one({"name": "send_credential_offer_to_device"})

    device = db.supplier_tb.find_one({"name": verinym_for["value"]}) #device1

    supplier = db.supplier_tb.find_one({"name": "Supplier"})

    print("\"Supplier\" -> Create \"Transcript\" Credential Offer for Alice")
    supplier['transcript_cred_offer'] = \
        await anoncreds.issuer_create_credential_offer(supplier['wallet'], supplier['supplier_certificate_cred_def_id'])

    offer = supplier['transcript_cred_offer']

    response = requests.post(f'http://{device}:5000/', data = {'offer': offer})
    supplier = response['entity']


if __name__ == '__main__':
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    await run()
    time.sleep(1)  # FIXME waiting for libindy thread complete
